# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [40]:
# import libraries
import datetime
import time

import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re

import nltk
nltk.download('punkt')

from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.tokenize import word_tokenize

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion, make_union

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.svm import SVC

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.multioutput import MultiOutputClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import  classification_report, confusion_matrix

from sklearn.externals import joblib

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/saha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/saha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/saha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/saha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')

In [3]:
df = pd.read_sql_table('InsertTableName',engine)

In [4]:
target_cols = df.columns.drop(['id', 'message', 'original', 'genre'])

In [5]:
X = df['message']
y = df[target_cols]

In [6]:
for col in target_cols:
    if len(y[col].unique())>2:
        print(y[col].unique())

[1 0 2]


In [7]:
y.related.replace(to_replace=2,value=1,inplace=True);

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    tokens = word_tokenize(text.lower().strip())
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    tokens = [PorterStemmer().stem(w) for w in tokens]
    return [WordNetLemmatizer().lemmatize(w) for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
transformer = make_union(CountVectorizer(tokenizer=tokenize))

In [10]:
clf = RandomForestClassifier()

In [11]:
pipeline = Pipeline([
    ('transformer',transformer),
    ('classifier',MultiOutputClassifier(clf,n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42)

In [13]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('transformer', FeatureUnion(n_jobs=1,
       transformer_list=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
  ...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
print(classification_report(y_pred=y_pred,y_true=y_test,target_names = target_cols))

                        precision    recall  f1-score   support

               related       0.77      0.89      0.82      4943
               request       0.39      0.10      0.16      1121
                 offer       0.00      0.00      0.00        31
           aid_related       0.46      0.25      0.32      2705
          medical_help       0.18      0.01      0.03       518
      medical_products       0.04      0.00      0.01       317
     search_and_rescue       0.00      0.00      0.00       189
              security       0.50      0.01      0.01       133
              military       0.00      0.00      0.00       237
           child_alone       0.00      0.00      0.00         0
                 water       0.15      0.01      0.02       426
                  food       0.16      0.01      0.03       732
               shelter       0.09      0.01      0.02       582
              clothing       0.00      0.00      0.00       104
                 money       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
clf = AdaBoostClassifier()

In [17]:
pipeline = Pipeline([
    ('transformer',transformer),
    ('classifier',MultiOutputClassifier(clf,n_jobs=-1))
])

In [18]:
parameters = [
    {
        "classifier__estimator": [AdaBoostClassifier()],
        "classifier__estimator__base_estimator":[
            DecisionTreeClassifier(),
            MultinomialNB()],
        "classifier__estimator__n_estimators": [25,35,45]
    },
    {
        "classifier__estimator": [RandomForestClassifier()],
        "classifier__estimator__n_estimators": [25,35,45]
    }
]

In [19]:
gridSearch = GridSearchCV(pipeline,parameters, verbose=2, n_jobs = 6)

In [20]:
best_model = gridSearch.fit(X_train,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), classifier__estimator__n_estimators=25 
[CV] classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0

            splitter='best'), classifier__estimator__n_estimators=35, total= 2.6min
[CV] classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), classifier__estimator__n_estimators=25 
[CV]  classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), classifier__estimator__n_estimators=25, total= 2.6min
[CV] classifier__estimator

            warm_start=False), classifier__estimator__n_estimators=25 
[CV] classifier__estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), classifier__estimator__n_estimators=35 
[CV]  classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), classifier__estimator__n_estimators=45, total= 2.7min
[CV] classifier__estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, ma

[Parallel(n_jobs=6)]: Done  27 out of  27 | elapsed: 19.4min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
y_pred = best_model.predict(X_test)

In [22]:
print(classification_report(y_pred=y_pred,y_true=y_test,target_names = target_cols))

                        precision    recall  f1-score   support

               related       0.76      0.99      0.86      4943
               request       0.44      0.09      0.15      1121
                 offer       0.00      0.00      0.00        31
           aid_related       0.50      0.10      0.17      2705
          medical_help       0.00      0.00      0.00       518
      medical_products       0.00      0.00      0.00       317
     search_and_rescue       0.00      0.00      0.00       189
              security       0.00      0.00      0.00       133
              military       0.00      0.00      0.00       237
           child_alone       0.00      0.00      0.00         0
                 water       0.67      0.00      0.01       426
                  food       0.40      0.01      0.02       732
               shelter       0.00      0.00      0.00       582
              clothing       0.25      0.01      0.02       104
                 money       0.00      

In [23]:
best_model.estimator

Pipeline(memory=None,
     steps=[('transformer', FeatureUnion(n_jobs=1,
       transformer_list=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
  ...ator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=-1))])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
transformer = make_union(TfidfVectorizer(tokenizer=tokenize))

In [27]:
pipeline = Pipeline([
    ('transformer',transformer),
    ('classifier',MultiOutputClassifier(clf,n_jobs=-1))
])

In [28]:
gridSearch = GridSearchCV(pipeline,parameters, verbose=2, n_jobs = -1)

In [29]:
best_model_refined = gridSearch.fit(X_train,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), classifier__estimator__n_estimators=25 
[CV] classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0

            splitter='best'), classifier__estimator__n_estimators=25, total= 3.6min
[CV] classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), classifier__estimator__n_estimators=25 
[CV]  classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), classifier__estimator__n_estimators=45, total= 3.6min
[CV] classifier__estimator

            warm_start=False), classifier__estimator__n_estimators=35 
[CV]  classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), classifier__estimator__base_estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), classifier__estimator__n_estimators=45, total= 3.6min
[CV] classifier__estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), classifier__estimator__n_estimators=35 
[CV]  classifier__estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50

            warm_start=False), classifier__estimator__n_estimators=45, total= 1.8min


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 18.7min finished


In [30]:
y_pred = best_model_refined.predict(X_test)

In [31]:
print(classification_report(y_pred=y_pred,y_true=y_test,target_names = target_cols))

                        precision    recall  f1-score   support

               related       0.76      0.97      0.85      4943
               request       0.47      0.12      0.18      1121
                 offer       0.00      0.00      0.00        31
           aid_related       0.49      0.16      0.25      2705
          medical_help       0.22      0.00      0.01       518
      medical_products       0.14      0.00      0.01       317
     search_and_rescue       0.00      0.00      0.00       189
              security       0.00      0.00      0.00       133
              military       0.12      0.00      0.01       237
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       426
                  food       0.18      0.00      0.01       732
               shelter       0.33      0.00      0.00       582
              clothing       0.60      0.03      0.06       104
                 money       0.00      

In [32]:
best_model.estimator

Pipeline(memory=None,
     steps=[('transformer', FeatureUnion(n_jobs=1,
       transformer_list=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
  ...ator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=-1))])

### 9. Export your model as a pickle file

In [43]:
timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

version = joblib.__version__

In [45]:
joblib.dump(best_model, "model_{version}_{timestamp}.pkl".format(version=version,timestamp = timestamp))

joblib.dump(best_model, "model_{version}.pkl".format(version=version,timestamp = timestamp))

['model_0.11.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.